In [5]:
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import numpy as np
import regex
import pandas
from progress.bar import Bar

# NEED TO NOTE OFF OF THE RIP, THIS PROGRAM IS CURRENTLY HEAVILY CUSTOMIZED FOR ADIMAB PNAS PAPER DATA
Sometime in the future, I should probably actually make it pretty customizable given any sequence

In [6]:
fullH=pandas.read_csv('seqsH.txt',header=None)
cdr3H=pandas.read_csv('cdr3H_germ.txt',header=None)
non3H=pandas.read_csv('non_cdr3H.txt',header=None)
alignmentH=pandas.read_csv('alignmentsH.txt',header=None)

fullL=pandas.read_csv('seqsL.txt',header=None)
cdr3L=pandas.read_csv('cdr3L_germ.txt',header=None)
non3L=pandas.read_csv('non_cdr3L.txt',header=None)
alignmentL=pandas.read_csv('alignmentsL.txt',header=None)

# HEAVY CHAIN

In [7]:
# Again, this is currently a bit of a hardcoded solution... Need to fix at some point.
# Should find out what the "find" wildcard is
for i in np.arange(len(alignmentH)):
    # Whole bigass block of code just for CDR1
    if alignmentH.values[i][0].find('<CDR1') != -1:
        start=alignmentH.values[i][0].find('<CDR1')
    if alignmentH.values[i][0].find('<-CDR1') != -1:
        start=alignmentH.values[i][0].find('<-CDR1')
    if alignmentH.values[i][0].find('<----FR2') != -1:
        end = alignmentH.values[i][0].find('<----FR2')
    elif alignmentH.values[i][0].find('<---FR2') != -1:
        end = alignmentH.values[i][0].find('<---FR2')
    cdr1_single=non3H.values[i][0][start:end-1]
    if i == 0:
        cdr1_final=cdr1_single
    else:
        cdr1_final=np.vstack((cdr1_final,cdr1_single))
        
     # Whole bigass block of code just for CDR2
    if alignmentH.values[i][0].find('<CDR2') != -1:
        start=alignmentH.values[i][0].find('<CDR2')
    if alignmentH.values[i][0].find('<--------------FR3') != -1:
        end = alignmentH.values[i][0].find('<--------------FR3')
    cdr2_single=non3H.values[i][0][start:end-1]
    if i == 0:
        cdr2_final=cdr2_single
    else:
        cdr2_final=np.vstack((cdr2_final,cdr2_single))
        
# EVIDENTLY AT THE END OF ALL THIS, WE CAN GET SOME WEIRD ALIGNMENT GAPS... SHOULD MAKE SURE 
# THAT I GET RID OF THOSE AT THE END OF ALL OF THIS

In [68]:
# LOAD IN THE HEAVY J-Gene
JgeneH=pandas.read_csv('human_IGHJ.txt',header=None)
# Take every other entry
heavyJ=JgeneH.iloc[::2, :].values[1:]

In [19]:
# Now we wanna get CDR3
full1 = fullH.where((pandas.notnull(fullH)), 'None')
full2 = fullH.where((pandas.notnull(full1)), [])

cdr3H1 = cdr3H.where((pandas.notnull(cdr3H)), 'None')
cdr3H2 = cdr3H.where((pandas.notnull(cdr3H1)), [])

cols=np.shape(full2)[1]
full_input=[Seq(a, IUPAC.protein) for a in full2.iloc[:,cols-1]]
full_cdr3H=[Seq(a, IUPAC.protein) for a in cdr3H2.iloc[:,cols-1]]

for i in np.arange(len(full_cdr3H)):
    # Select each single sequence
    test1=full_cdr3H[i]
    test2=full_input[i]
    # Do the alignments
    # This localxs alignment should not allow gaps, with this below comment
    aa=pairwise2.align.localxs(test1,test2,-1,-1)
    #### IMPORTANT NOTE #######
    # So for now, this is ok, but if there are EVER multiple Cysteines in these final stretches,
    # this "find" command will fail. Keep that in mind
    where=aa[0][0].find('C')
    cdr3H_single=aa[0][1][where+1:]
    
    if i==0:
        cdr3H_final=cdr3H_single
    else:
        cdr3H_final=np.vstack((cdr3H_final,cdr3H_single))
        
for i in np.arange(len(cdr3H_final)):
    for j in np.arange(len(heavyJ)):
        test1=cdr3H_final[i][0]
        test2=heavyJ[j][0]
        aa=pairwise2.align.localxs(test1,test2,-1,-1)
        if j==0:
            best=aa[0][2]
            Jnum=j
        else:
            if aa[0][2] > best:
                best=aa[0][2]
                Jnum=j
        align_final=pairwise2.align.localxs(test1,heavyJ[Jnum],-1,-1)
        where=align_final[0][3]
        short_3H=aa[0][0][0:where]
        if i==0:
            cdr3H_final_jmin=short_3H
        else:
            cdr3H_final_jmin=np.vstack((cdr3H_final_jmin,short_3H))

In [9]:
final=np.hstack((cdr1_final,cdr2_final,cdr3H_final_jmin))
np.savetxt('cdrs_H.txt',final,fmt='%s')

# LIGHT CHAIN

In [48]:
# Again, this is currently a bit of a hardcoded solution... Need to fix at some point.
# Should find out what the "find" wildcard is
for i in np.arange(len(alignmentL)):
    # Whole bigass block of code just for CDR1
    if alignmentL.values[i][0].find('<CDR1') != -1:
        start=alignmentL.values[i][0].find('<CDR1')
    if alignmentL.values[i][0].find('<-CDR1') != -1:
        start=alignmentL.values[i][0].find('<-CDR1')
    if alignmentL.values[i][0].find('<---FR2') != -1:
        end = alignmentL.values[i][0].find('<---FR2')
    elif alignmentL.values[i][0].find('<-----FR2') != -1:
        end = alignmentL.values[i][0].find('<-----FR2')
    cdr1_single=non3L.values[i][0][start:end-1]
    if i == 0:
        cdr1_final=cdr1_single
    else:
        cdr1_final=np.vstack((cdr1_final,cdr1_single))
        
     # Whole bigass block of code just for CDR2
    if alignmentL.values[i][0].find('<C>') != -1:
        start=alignmentL.values[i][0].find('<C>')
    if alignmentL.values[i][0].find('<-------------FR3') != -1:
        end = alignmentL.values[i][0].find('<-------------FR3')
    elif alignmentL.values[i][0].find('<--------------FR3') != -1:
        end = alignmentL.values[i][0].find('<--------------FR3')
    cdr2_single=non3L.values[i][0][start:end]
    if i == 0:
        cdr2_final=cdr2_single
    else:
        cdr2_final=np.vstack((cdr2_final,cdr2_single))
        
# EVIDENTLY AT THE END OF ALL THIS, WE CAN GET SOME WEIRD ALIGNMENT GAPS... SHOULD MAKE SURE 
# THAT I GET RID OF THOSE AT THE END OF ALL OF THIS

In [87]:
# LOAD IN THE HEAVY J-Gene
JgeneL1=pandas.read_csv('human_IGKJ.txt',header=None)
# Take every other entry
lightJ1=JgeneL1.iloc[::2, :].values[1:]
JgeneL2=pandas.read_csv('human_IGLJ.txt',header=None)
# Take every other entry
lightJ2=JgeneL2.iloc[::2, :].values[1:]

lightJ=np.vstack((lightJ1,lightJ2))

In [90]:
# Now we wanna get CDR3
full1 = fullL.where((pandas.notnull(fullL)), 'None')
full2 = fullL.where((pandas.notnull(full1)), [])

cdr3L1 = cdr3L.where((pandas.notnull(cdr3L)), 'None')
cdr3L2 = cdr3L.where((pandas.notnull(cdr3L1)), [])

cols=np.shape(full2)[1]
full_input=[Seq(a, IUPAC.protein) for a in full2.iloc[:,cols-1]]
full_cdr3L=[Seq(a, IUPAC.protein) for a in cdr3L2.iloc[:,cols-1]]

for i in np.arange(len(full_cdr3L)):
    # Select each single sequence
    test1=full_cdr3L[i]
    test2=full_input[i]
    # Do the alignments
    # This localxs alignment should not allow gaps, with this below comment
    aa=pairwise2.align.localxs(test1,test2,-1,-1)
    #### IMPORTANT NOTE #######
    # So for now, we don't have that same neat conserved cysteine in the CDR3 germline region,
    # but there does seem to be a conserved, single YC or FC we can just grab.
    # NOTE THERE IS A SINGLE HC... UNSURE IF THIS IS CONSERVED
    where=aa[0][1].find('YC')
    if where == -1:
        where=aa[0][1].find('FC')
        if where == -1:
            where=aa[0][1].find('HC')
    # Have to add 2 because we're doing a two-character "find" and it starts from first character
    cdr3L_single=aa[0][1][where+2:]
    
    if i==0:
        cdr3L_final=cdr3L_single
    else:
        cdr3L_final=np.vstack((cdr3L_final,cdr3L_single))
        
for i in np.arange(len(cdr3L_final)):
    for j in np.arange(len(lightJ)):
        test1=cdr3L_final[i][0]
        test2=lightJ[j][0]
        aa=pairwise2.align.localxs(test1,test2,-1,-1)
        if j==0:
            best=aa[0][2]
            Jnum=j
        else:
            if aa[0][2] > best:
                best=aa[0][2]
                Jnum=j
        align_final=pairwise2.align.localxs(test1,lightJ[Jnum],-1,-1)
        where=align_final[0][3]
        short_3L=aa[0][0][0:where]
    if i==0:
        cdr3L_final_jmin=short_3L
    else:
            cdr3L_final_jmin=np.vstack((cdr3L_final_jmin,short_3L))

In [97]:
cdr3H_final_jmin[800]

array(['ARSAYYDYDGFAYW'], dtype='<U24')

In [95]:
np.shape(cdr3H_final_jmin)

(817, 1)

In [93]:
final=np.hstack((cdr1_final,cdr2_final,cdr3L_final_jmin))
np.savetxt('cdrs_L.txt',final,fmt='%s')

ValueError: all the input array dimensions except for the concatenation axis must match exactly